# Export a model


In this section of the lab you will export the trained model and learn how to use it in your application. 
Custom Vision Service allows classifiers to be exported to run offline. You can embed your exported classifier into an application and run it locally on a device for real-time classification.

Custom Vision Service supports the following exports:

- Tensorflow for Android.
- CoreML for iOS11.
- ONNX for Windows ML.
- A Windows or Linux container. The container includes a Tensorflow model and service code to use the Custom Vision Service API.

Custom Vision Service only exports compact domains. The models generated by compact domains are optimized for the constraints of real-time classification on low powered devices. Classifiers built with a compact domain may be slightly less accurate than a standard domain with the same amount of training data.



## Retrain with a compact domain
If your classifier was not trained with the compact domain (which is the case for the classifier trained during the first stage of the lab) you need to retrain it.

In [ ]:
training_key = 'b1cbbf0f9a054ef481113c9efec1fe4d'
prediction_key = 'b01851d0bbe24e9ba3c542ce84306787'
project_name = 'AerialClassifier'

In [2]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

trainer = training_api.TrainingApi(training_key)

### Change the project's domain

In [6]:
# Find you project's ID
project_id = None
for project in trainer.get_projects():
    if project.name == project_name:
        project_id = project.id
        print("Found project: {0}".format(project_id))
        break
        
if project_id == None:
    print("Could not find your project")


Could not find your project


In [ ]:
import time

def train(training_key, project_id, domain_id):
    trainer = training_api.TrainingApi(training_key)
    print("Starting training...")
    try:
        iteration = trainer.train_project(project.id)
    except:
        print("No need to retrain")
        return
    
    while (iteration.status != "Completed"):
        iteration = trainer.get_iteration(project.id, iteration.id)
        print ("Training status: " + iteration.status)
        time.sleep(2)

    # The iteration is now trained. Make it the default project endpoint
    trainer.update_iteration(project_id, iteration.id, is_default=True)
    print("Done")
    return iteration.id

## Export the iteration
Exporting a model is a two-step process. First you must request the export. It is an asynchronous call. You have to periodically check the status of the request and when the export package is ready you can download it using the returned URI.


Navigate to **Project Settings** and copy and paste *Project ID* and *Training Key*. Also, cut and paste the name of the iteration which was trained with the compact domain.


In [ ]:
PROJECT_ID = '<your project id>'
TRAINING_ID = '<your training key>'
ITERATION_NAME = '<your iteration name>'

### Request the export

In [ ]:
import azure.cognitiveservices.vision.customvision.training as training_api


trainer = training_api.TrainingApi(TRAINING_KEY)

# Search through iterations collection to find the one you want to export
iteration_id = None
for iteration in trainer.get_iterations(PROJECT_ID):
    if iteration.name == ITERATION_NAME:
        iteration_id = iteration.id
        break

# Request the export
if iteration_id == None:
    print("Could not find the iteration")
else:
    print("Requesting export for Iteration ID: {0}".format(iteration_id))
    try:
        trainer.export_iteration(PROJECT_ID, iteration_id, platform='DockerFile', flavor='Linux')
    except:
        print("Error while requesting export. Most likely the export already exists")
    

### Wait till the export is ready


In [ ]:
import time

# Periodically check the status of request
download_uri = None
while True:
    exports = trainer.get_exports(PROJECT_ID, iteration_id)
    # Since we have only submitted one request check 
    # the first request on the list
    if exports[0].status == 'Done':
        print("Export succeded")
        download_uri = exports[0].download_uri
        break
    elif exports[0].status == 'Failed':
        print("Export failed")
        break
    # Wait a little bit
    print("Going to sleep for a few seconds")
    time.sleep(10)
        

### Download the package

In [ ]:
import wget
import os

download_filename = 'docker_export.zip'

if download_uri != None:
    print("Downloading from: {0}".format(download_uri))
    wget.download(download_uri, download_filename)

In [ ]:
%%sh
ls -l 

In [ ]:
%%sh
unzip docker_export.zip -d docker_export

In [ ]:
%%sh
ls -l docker_export

### Review the exported package
Your instructor will guide you through the content of the exported package


## Create and test docker image

As explained in more detail by the instructor, the exported package includes a pre-configured *DockerFile*. You are free to modify it but in this lab we will use it as it is. 


### Create docker image 


In [ ]:
%%sh
cd docker_export
sudo docker build . -t aerialclassifier
sudo docker images

We will now test the image in the local container.

### Start docker container

In [ ]:
%%sh
sudo docker run -p 127.0.0.1:80:80 -d aerialclassifier
sudo docker ps

### Invoke the web service in the container

In [ ]:
%%sh
curl -X POST http://127.0.0.1/image -F imageData=@samples/developed-1.png

Your container is tested and ready for deployment. You can deploy it to the runtime of your choice, including:
- Azure Kubernetis Service
- IoT Edge, or
- Any other docker runtime
